# Epistasis data aggregation

Build **one epistasis dataframe** from all structured subset CSVs, with a **`source`** column indicating the analysis/source. This file is intended to be fed into the pipeline that uses `add_epistasis_metrics` (or equivalent):

- **Downstream usage:** Variants from the **same source** are processed together and stored in **their own database directory**, e.g. `embeddings/kras/`, `embeddings/fas/`, each containing one SQLite DB per tool (e.g. `nt100_multi.db`, `borzoi.db`). So you group the aggregated table by `source`, and for each source run the embedding + metrics pipeline writing to `embeddings/{source}/`.

In [ ]:
import os
import sys
import pandas as pd
from pathlib import Path

# Ensure repo root is on path so "notebooks" can be imported
ROOT = Path.cwd()
for _ in range(4):
    if (ROOT / "notebooks" / "paper_data_config.py").exists():
        break
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# All data under paper root (set EPISTASIS_PAPER_ROOT env to override)
from notebooks.paper_data_config import data_dir
DATA_DIR = data_dir()  # EPISTASIS_PAPER_ROOT / "data"

# Output: single aggregated file in DATA_DIR
OUT_AGGREGATED = "epistasis_aggregated.csv"

# (filename or path, source label). resolve_path looks in DATA_DIR, then parent, then cwd.
SUBSETS = [
    ("kras_neighborhood_doubles.csv", "kras"),
    ("fas_subset.csv", "fas"),
    ("mst1r_subset.csv", "mst1r"),
    ("cftr_folding_pairs.csv", "cftr_folding"),
    ("okgp_epistasis.csv", "okgp"),
    ("okgp_doubles_subset.csv", "okgp"),  # fallback if okgp_epistasis not present
    ("tcga_subset_doubles.csv", "tcga"),
    ("topld_doubles_subset_clean.csv", "topld"),
    ("lt6000_eqtl_connected.csv", "eqtl_functional"),
    ("correlated_variants_27K.csv", "eqtl_null"),
    ("mrna_folding_epistasis.csv", "mrna_folding"),  # from mrna_folding_epistasis.ipynb
]




## Load each subset and add `source`

In [ ]:
def resolve_path(name):
    for base in [Path(DATA_DIR), Path(DATA_DIR).parent, Path(".")]:
        p = base / name
        if p.is_file():
            return str(p)
    return None

def load_subset(path, source_label):
    df = pd.read_csv(path)
    if "epistasis_id" not in df.columns:
        if "mut1" in df.columns and "mut2" in df.columns:
            df["epistasis_id"] = df["mut1"].astype(str) + "|" + df["mut2"].astype(str)
        else:
            raise ValueError(f"No epistasis_id or mut1/mut2 in {path}")
    df["source"] = source_label
    return df

seen_sources = set()
frames = []
for name, source_label in SUBSETS:
    path = resolve_path(name)
    if path is None:
        continue
    # Avoid loading same source twice (e.g. okgp_epistasis and okgp_doubles_subset)
    if source_label in seen_sources:
        continue
    try:
        df = load_subset(path, source_label)
        frames.append(df)
        seen_sources.add(source_label)
        print(f"Loaded {len(df)} rows from {name} -> source={source_label}")
    except Exception as e:
        print(f"Skip {name}: {e}")
        continue

if not frames:
    raise SystemExit("No subset files found. Check DATA_DIR and SUBSETS paths.")


    

## Concatenate and normalize columns

In [ ]:
# Keep at least epistasis_id and source; optionally keep common columns if present
COMMON_COLS = ["epistasis_id", "source", "mut1", "mut2", "gene", "chrom", "pos1", "pos2", "distance"]

agg = pd.concat(frames, ignore_index=True)
# Ensure required columns
assert agg["epistasis_id"].notna().all(), "epistasis_id must be non-null"
agg["epistasis_id"] = agg["epistasis_id"].astype(str)

# Optional: restrict to common columns that exist (so schema is consistent)
cols_to_keep = [c for c in COMMON_COLS if c in agg.columns]
agg = agg[cols_to_keep].copy()

agg = agg.drop_duplicates(subset=["epistasis_id", "source"])
agg = agg.sort_values(["source", "epistasis_id"]).reset_index(drop=True)

print("Shape:", agg.shape)
print(agg.groupby("source").size())

## Save aggregated file

In [ ]:
out_path = Path(DATA_DIR) / OUT_AGGREGATED
agg.to_csv(out_path, index=False)
print(f"Saved {len(agg)} rows to {out_path}")
agg.head(10)

---
## Downstream: process by source and store per-source DBs

Use the aggregated file so that **each source** is processed and written to its **own directory** of SQLite DBs (one per tool):

```python
import pandas as pd
from genebeddings.genebeddings import add_epistasis_metrics, VariantEmbeddingDB

agg = pd.read_csv("epistasis_aggregated.csv")
BASE_DB_DIR = "embeddings"  # or your root for tool DBs

for source, sub_df in agg.groupby("source"):
    out_dir = os.path.join(BASE_DB_DIR, source)
    os.makedirs(out_dir, exist_ok=True)
    # For each tool, db path = out_dir / f"{tool_name}.db"
    # db = VariantEmbeddingDB(os.path.join(out_dir, "nt100_multi.db"))
    # new_data = add_epistasis_metrics(sub_df, db, id_col="epistasis_id", ...)
```

Resulting layout: `embeddings/kras/nt100_multi.db`, `embeddings/kras/borzoi.db`, `embeddings/fas/nt100_multi.db`, etc.